import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [4]:
pip install pmdarima

   ---------------------------------------- 0.0/614.7 kB ? eta -:--:--
   --------------------------------------  614.4/614.7 kB 19.5 MB/s eta 0:00:01
   --------------------------------------- 614.7/614.7 kB 12.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --------------------------- ------------ 1.9/2.8 MB 61.1 MB/s eta 0:00:01
   ---------------------------------------  2.8/2.8 MB 59.0 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 35.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ------ --------------------------------- 1.7/9.9 MB 36.2 MB/s eta 0:00:01
   -------------- ------------------------- 3.6/9.9 MB 38.5 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.9 MB 38.0 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.9 MB 28.4 MB/s eta 0:00:01
   -------------------------------- ------- 8.0/9.9 MB 31.9 MB/s eta 0:00:01
   ------------

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Load the dataset
file_path = 'phase1_only_records.csv'
data = pd.read_csv(file_path)

# Function to create sequences for CNN-LSTM model
def create_sequences(data, seq_length=24):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length][['Demand_kWh', 'Temperature']].values
        target = data.iloc[i+seq_length]['Demand_kWh']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Define CNN-LSTM model
def create_cnn_lstm_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        LSTM(50, activation='relu', return_sequences=True),
        LSTM(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Preprocess the data
def preprocess_data(data):
    # Ensure the data is sorted by ID and Date
    data = data.sort_values(by=['Date', 'Hour']).reset_index(drop=True)
    
    # Normalize the temperature and demand columns
    temp_columns = ['Temperature']
    data[temp_columns] = (data[temp_columns] - data[temp_columns].mean()) / data[temp_columns].std()
    data['Demand_kWh'] = (data['Demand_kWh'] - data['Demand_kWh'].mean()) / data['Demand_kWh'].std()

    return data

# Split the data into train and test sets
def split_data(data, test_size=0.2):
    train_data = {}
    test_data = {}

    ids = data['ID'].unique()
    
    for id_ in ids:
        id_data = data[data['ID'] == id_]
        train, test = train_test_split(id_data, test_size=test_size, shuffle=False)
        train_data[id_] = train
        test_data[id_] = test
    
    return train_data, test_data

# Preprocess the dataset
data = preprocess_data(data)

# Split the data into train and test sets
train_data, test_data = split_data(data)

# Initialize lists to store results
results = []

# Train and evaluate model for each ID
for id_ in train_data.keys():
    # Prepare training and testing data
    train_sequences, train_targets = create_sequences(train_data[id_])
    test_sequences, test_targets = create_sequences(test_data[id_])
    
    # Reshape input data for the CNN-LSTM model
    input_shape = (train_sequences.shape[1], train_sequences.shape[2])
    train_sequences = train_sequences.reshape((train_sequences.shape[0], train_sequences.shape[1], train_sequences.shape[2]))
    test_sequences = test_sequences.reshape((test_sequences.shape[0], test_sequences.shape[1], test_sequences.shape[2]))
    
    # Create and train the model
    model = create_cnn_lstm_model(input_shape)
    model.fit(train_sequences, train_targets, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
    
    # Predict on the test data
    predictions = model.predict(test_sequences)
    
    # Inverse normalize the predictions and actual values
    demand_mean = data['Demand_kWh'].mean()
    demand_std = data['Demand_kWh'].std()
    predictions = predictions * demand_std + demand_mean
    test_targets = test_targets * demand_std + demand_mean
    
    # Calculate metrics
    mse = mean_squared_error(test_targets, predictions)
    mae = mean_absolute_error(test_targets, predictions)
    rmse = math.sqrt(mse)
    
    # Store results
    for i in range(len(test_targets)):
        results.append([id_, test_data[id_].iloc[i + 24]['Date'], test_data[id_].iloc[i + 24]['Hour'], test_targets[i], predictions[i][0]])

# Save predictions to a CSV file
results_df = pd.DataFrame(results, columns=['ID', 'Date', 'Hour', 'Actual_Demand', 'Predicted_Demand'])
results_df.to_csv('cnnlstm_results_1_8020.csv', index=False)

# Calculate overall metrics
overall_mse = mean_squared_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_mae = mean_absolute_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_rmse = math.sqrt(overall_mse)

print(f"Overall Test Loss (MSE): {overall_mse}")
print(f"Overall Test Loss (MAE): {overall_mae}")
print(f"Overall Test Loss (RMSE): {overall_rmse}")


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step
Overall Test Loss (MSE): 0.20774002008340414
Overall Test Loss (MAE): 0.2958146098549286
Overall Test Loss (RMSE): 0.45578505908312106


In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Load the dataset
file_path = 'phase1_only_records.csv'
data = pd.read_csv(file_path)

# Function to create sequences for CNN-LSTM model
def create_sequences(data, seq_length=24):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length][['Demand_kWh', 'Temperature']].values
        target = data.iloc[i+seq_length]['Demand_kWh']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Define CNN-LSTM model
def create_cnn_lstm_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        LSTM(50, activation='relu', return_sequences=True),
        LSTM(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Preprocess the data
def preprocess_data(data):
    # Ensure the data is sorted by ID and Date
    data = data.sort_values(by=['Date', 'Hour']).reset_index(drop=True)
    
    # Normalize the temperature and demand columns
    temp_columns = ['Temperature']
    data[temp_columns] = (data[temp_columns] - data[temp_columns].mean()) / data[temp_columns].std()
    data['Demand_kWh'] = (data['Demand_kWh'] - data['Demand_kWh'].mean()) / data['Demand_kWh'].std()

    return data

# Split the data into train and test sets
def split_data(data, test_size=0.2):
    train_data = {}
    test_data = {}

    ids = data['ID'].unique()
    
    for id_ in ids:
        id_data = data[data['ID'] == id_]
        train, test = train_test_split(id_data, test_size=test_size, shuffle=False)
        train_data[id_] = train
        test_data[id_] = test
    
    return train_data, test_data

# Preprocess the dataset
data = preprocess_data(data)

# Split the data into train and test sets
train_data, test_data = split_data(data)

# Initialize lists to store results
results = []

# Train and evaluate model for each ID
for id_ in train_data.keys():
    # Prepare training and testing data
    train_sequences, train_targets = create_sequences(train_data[id_])
    test_sequences, test_targets = create_sequences(test_data[id_])
    
    # Reshape input data for the CNN-LSTM model
    input_shape = (train_sequences.shape[1], train_sequences.shape[2])
    train_sequences = train_sequences.reshape((train_sequences.shape[0], train_sequences.shape[1], train_sequences.shape[2]))
    test_sequences = test_sequences.reshape((test_sequences.shape[0], test_sequences.shape[1], test_sequences.shape[2]))
    
    # Create and train the model
    model = create_cnn_lstm_model(input_shape)
    model.fit(train_sequences, train_targets, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
    
    # Predict on the test data
    predictions = model.predict(test_sequences)
    
    # Inverse normalize the predictions and actual values
    demand_mean = data['Demand_kWh'].mean()
    demand_std = data['Demand_kWh'].std()
    predictions = predictions * demand_std + demand_mean
    test_targets = test_targets * demand_std + demand_mean
    
    # Store results
    for i in range(len(test_targets)):
        results.append([id_, test_data[id_].iloc[i + 24]['Date'], test_data[id_].iloc[i + 24]['Hour'], test_data[id_].iloc[i + 24]['Demand_kWh'], predictions[i][0]])

# Save predictions to a CSV file
results_df = pd.DataFrame(results, columns=['ID', 'Date', 'Hour', 'Actual_Demand', 'Predicted_Demand'])
results_df.to_csv('lstm_demand_predictions.csv', index=False)

# Calculate overall metrics
overall_mse = mean_squared_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_mae = mean_absolute_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_rmse = math.sqrt(overall_mse)

print(f"Overall Test Loss (MSE): {overall_mse}")
print(f"Overall Test Loss (MAE): {overall_mae}")
print(f"Overall Test Loss (RMSE): {overall_rmse}")


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step
Overall Test Loss (MSE): 0.2099656627237301
Overall Test Loss (MAE): 0.30331408928045545
Overall Test Loss (RMSE): 0.4582201029240534


In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Load the dataset
file_path = 'phase1_only_records.csv'
data = pd.read_csv(file_path)

# Function to create sequences for CNN-LSTM model
def create_sequences(data, seq_length=24):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length][['Demand_kWh', 'Temperature']].values
        target = data.iloc[i+seq_length]['Demand_kWh']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Define CNN-LSTM model
def create_cnn_lstm_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        LSTM(50, activation='relu', return_sequences=True),
        LSTM(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Preprocess the data
def preprocess_data(data):
    # Ensure the data is sorted by ID and Date
    data = data.sort_values(by=['From']).reset_index(drop=True)
    
    # Normalize the temperature and demand columns
    temp_columns = ['Temperature']
    data[temp_columns] = (data[temp_columns] - data[temp_columns].mean()) / data[temp_columns].std()
    data['Demand_kWh'] = (data['Demand_kWh'] - data['Demand_kWh'].mean()) / data['Demand_kWh'].std()

    return data

# Split the data into train and test sets
def split_data(data, test_size=0.2):
    train_data = {}
    test_data = {}

    ids = data['ID'].unique()
    
    for id_ in ids:
        id_data = data[data['ID'] == id_]
        train, test = train_test_split(id_data, test_size=test_size, shuffle=False)
        train_data[id_] = train
        test_data[id_] = test
    
    return train_data, test_data

# Preprocess the dataset
data = preprocess_data(data)

# Split the data into train and test sets
train_data, test_data = split_data(data)

# Initialize lists to store results
results = []

# Train and evaluate model for each ID
for id_ in train_data.keys():
    # Prepare training and testing data
    train_sequences, train_targets = create_sequences(train_data[id_])
    test_sequences, test_targets = create_sequences(test_data[id_])
    
    # Reshape input data for the CNN-LSTM model
    input_shape = (train_sequences.shape[1], train_sequences.shape[2])
    train_sequences = train_sequences.reshape((train_sequences.shape[0], train_sequences.shape[1], train_sequences.shape[2]))
    test_sequences = test_sequences.reshape((test_sequences.shape[0], test_sequences.shape[1], test_sequences.shape[2]))
    
    # Create and train the model
    model = create_cnn_lstm_model(input_shape)
    model.fit(train_sequences, train_targets, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
    
    # Predict on the test data
    predictions = model.predict(test_sequences)
    
    # Inverse normalize the predictions and actual values
    demand_mean = data['Demand_kWh'].mean()
    demand_std = data['Demand_kWh'].std()
    predictions = predictions * demand_std + demand_mean
    test_targets = test_targets * demand_std + demand_mean
    
    # Store results
    for i in range(len(test_targets)):
        actual_demand = test_data[id_].iloc[i + 24]['Demand_kWh'] * demand_std + demand_mean
        results.append([id_, test_data[id_].iloc[i + 24]['Date'], test_data[id_].iloc[i + 24]['Hour'], actual_demand, predictions[i][0]])

# Save predictions to a CSV file
results_df = pd.DataFrame(results, columns=['ID', 'Date', 'Hour', 'Actual_Demand', 'Predicted_Demand'])
results_df.to_csv('ccn1_predictions.csv', index=False)

# Calculate overall metrics
overall_mse = mean_squared_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_mae = mean_absolute_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_rmse = math.sqrt(overall_mse)

print(f"Overall Test Loss (MSE): {overall_mse}")
print(f"Overall Test Loss (MAE): {overall_mae}")
print(f"Overall Test Loss (RMSE): {overall_rmse}")


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 126ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step
Overall Test Loss (MSE): 0.24023846274245184
Overall Test Loss (MAE): 0.3236550928988239
Overall Test Loss (RMSE): 0.4901412681487367
